# Value Iteration

* 목표: optimal policy $\pi$ 찾기
* 해법: Bellman optimality backup을 반복적으로 적용
* $v_1 \rightarrow v_2 \rightarrow \cdots \rightarrow v_*$
* Synchronous backpu 사용
  * 각각의 iteration $k+1$ 에서
  * 모든 state $s\in S$ 에 대해
  * $v_k (s')$ 로부터 $v_{k+1}(s)$ 를 업뎃
* $v_*$ 가 수렴한다는 것은 증명되어 있음
* Policy iteration 과 달리, explicity 한 policy 없음
* 중간 단계의 value function 은 어떤 policy 에 관한 것도 아닐 수 있음

\begin{align}
v_{k+1} (s) & = \max_{a\in A}\Big( R_s^a + \gamma \sum_{s' \in S} P^a_{ss'} v_k \big(s' \big) \Big)\\
v_{k+1} & = \max_{a\in A} R^a + \gamma P^a v_k
\end{align}

## Frozen Lake
* Frozen lake 환경을 세팅
* Gym 에서 discrete 한 environment 쓸만한 놈이 이거라서 가져옴
* 'S' 타일에서 시작
* Action으로 'Up', 'Left', 'Right', 'Down'을 줄 수 있음
* 에피소드는 'H' 타일에 빠져서 reward 0 (죽음) 얻거나, 'G' (goal) 타일로 가서 reward 1 (점수) 얻거나

In [1]:
import gym
import numpy as np

```is_slippery``` 를 True, False에 따라
* True: action에 따라 확정적으로 이동하지 않고, 확률적으로 action 말고 다른데로 가버림 (frozen lake라 미끄러워서 그렇다고 함...)
* False: action 에 따라 확정적으로 이동

In [2]:
env = gym.make('FrozenLake-v0', is_slippery=False)

In [3]:
S = env.observation_space.n
S_ = env.observation_space.n
A = env.action_space.n
print('state S space:', np.arange(S))
print('state S\' space:', np.arange(S_))
print('action A space:', np.arange(A))

state S space: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
state S' space: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
action A space: [0 1 2 3]


In [4]:
print("랜덤하게 움직여서 겜 한판 돌려보자!")
s = env.reset()
env.render()


while True:
    s, r, done, _ = env.step(env.action_space.sample())
    env.render()
    if done:
        break

랜덤하게 움직여서 겜 한판 돌려보자!

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


## constant
* $\gamma$ : discount factor
* $\theta$ : policy evaluation threshold

In [5]:
GAMMA = .9 # discount factor
EPISODES = 1000 # policy evaluation 끝나고 transition matrix 업데이트 할 횟수
THETA = 1e-4 # policy evaluation threshold

## variables

* $\pi$ : policy. state에 따라 어떤 action 취할지 정함

In [6]:
# init policy, S x A x 1
pi = np.ones((S, A))
pi /= A
pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

* 리워드

In [7]:
# init rewrads, S x A x 1
rewards = np.zeros((S, A))

* transition (model 이라고도 함)

In [8]:
# init transition matrix, A x S x S' x 1
Pss = np.zeros((A, S, S_))

* value function : 이 예제에서는 state-value function 을 다룸

In [9]:
# init value function, S x 1
valuefunction = np.ones((S)) / S

In [10]:
# return list
# return_list = []

### Value Iteration, for estimating $\pi \approx \pi_*$

Algorithm parameter: 정확도 estimation 을 위한 threshold $\theta>0$<br/>
모든 $s\in S^+$에 대한 $V(s)$를 임의의 값으로 초기화, 단 $V(terminal)=0$으로<br/><br/>
Loop:</br>
|&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow 0$</br>
|&nbsp;&nbsp;&nbsp;&nbsp;Loop for each $s\in S$:</br>
|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$v \leftarrow V(s)$</br>
|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$V(s) \leftarrow \max_a \sum_{s', r}p(s',r|s,a)[r+\gamma V(s')]$</br>
|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow \max (\Delta, |v - V(s)|)$</br>
until $\Delta < \theta$<br/><br/>
아래를 만족하는 deterministic policy, $\pi \approx \pi_*$ 를 구함</br>
&nbsp;&nbsp;&nbsp;&nbsp;$\pi (s) = \arg \max_a \sum_{s', r}p(s', r|s, a)[r+\gamma V(s')]$

In [11]:
action = np.arange(A)

def get_action(state):
    """
    args:
        state: 상태
    return: 
        policy 의 확률에 따라 취할 수 있는 action 중에서 하나 구해서 리턴
    """
    return np.random.choice(action, size=None, p=pi[state])

def get_transition(episodes):
    """
    args:
        episodes 수
    return: 
        None
        에피소드 수 주어진 만큼 environment 돌아다니면서 transition matrix 채움
    """
    global rewards, return_list
    
    _rewards = np.zeros(rewards.shape)
    
    for _ in range(episodes):
        state = env.reset()
        while True:
            action = get_action(state)
            state_, _reward, done, _ = env.step(action)
            
            Pss[action][state][state_] += 1 # cumulate again and again
            _rewards[state][action] += _reward
            
            if done:
                break
            
            state = state_
                
    # average rewards
    _rewards /= episodes
    rewards = _rewards
#     return_list.append(np.sum(rewards))

아래 구현은 

$$
v_{k+1}(s) = \max_{a\in A} \big( R_s^a + \gamma \sum_{s' \in S} P^a_{ss'} v_k (s') \big)
$$

요걸로 함

In [12]:
# VALUE ITERATION

# 에피소드 수 만큼 돌면서 transition matrix를 채움
get_transition(episodes=100)

while True:
    # loop 멈출 현재의 threshold
    delta = .0
    
    pss_sum = np.sum(Pss)
    _Pss = Pss/pss_sum # 수정해야함
    PssV = np.matmul(_Pss, valuefunction)
    PssV *= GAMMA # (A,S)
    
    _valuefunction = PssV.T + rewards
    
    newV = np.zeros(valuefunction.shape)
    
    for s in range(S):
        newV[s] = np.max(_valuefunction[s])
            
    delta = max(delta, np.sum(np.abs(valuefunction - newV)))
    
    valuefunction = np.array(newV)

    if delta < THETA:
        break

    # 그 담번엔 EPISODES번씩
    get_transition(EPISODES)

아래를 만족하는 deterministic policy, $\pi \approx \pi_*$ 를 구함</br>
&nbsp;&nbsp;&nbsp;&nbsp;$\pi (s) = \arg \max_a \sum_{s', r}p(s', r|s, a)[r+\gamma V(s')]$

In [13]:
print('pi (S, A) =', pi.shape, ', Pss (A, S, S\') =', Pss.shape, ', reward (S, A)', rewards.shape)

pi (S, A) = (16, 4) , Pss (A, S, S') = (4, 16, 16) , reward (S, A) (16, 4)


In [14]:
# argmax_a 만족하는 a만 1이고, 나머지는 0 되도록 하기 위해서
pi = np.zeros(pi.shape)

In [15]:
pss_sum = np.sum(Pss)
_Pss = Pss/pss_sum
PssV = np.matmul(_Pss, valuefunction)
PssV *= GAMMA

_valuefunction = PssV.T + rewards

In [16]:
for s in range(S):
    pi[s][np.argmax(_valuefunction[s])] = 1.

In [17]:
pi

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]])

In [18]:
print("Policy iteration으로 찾은 policy에 따라 겜 해보자!")
s = env.reset()
env.render()

while True:
    s, r, done, _ = env.step(get_action(s))
    env.render()
    if done:
        break
        
print('reward:',r)

Policy iteration으로 찾은 policy에 따라 겜 해보자!

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
reward: 1.0


In [19]:
# for a in range(A):
#     for s in range(S):
#         for s_ in range(S):
#             print(f"(a:{a}, s:{s}) -> s':{s_} : {Pss[a][s][s_]}")

In [20]:
s = env.reset()
cnt_success = 0
trial = 1000

for _ in range(trial):
    while True:
        s, r, done, _ = env.step(get_action(s))
        cnt_success += r
        if done:
            s = env.reset()
            break
        
print(f"policy iteration 으로 풀었을 때, 에이전트 퍼포먼스: {trial}회 중 {cnt_success:n}회 성공 -> {cnt_success/trial * 100:.2f}%")

policy iteration 으로 풀었을 때, 에이전트 퍼포먼스: 1000회 중 1000회 성공 -> 100.00%


In [21]:
s = env.reset()
cnt_success = 0
trial = 1000

for _ in range(trial):
    while True:
        s, r, done, _ = env.step(env.action_space.sample())

        cnt_success += r

        if done:
            s = env.reset()
            break
        
print(f"아무곳이나 가는 에이전트 퍼포먼스: {trial}회 중 {cnt_success:n}회 성공 -> {cnt_success/trial * 100:.2f}%")

아무곳이나 가는 에이전트 퍼포먼스: 1000회 중 13회 성공 -> 1.30%
